<a href="https://colab.research.google.com/github/freguti/progettoML/blob/main/ProgettoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP:

1 - Dovremo prendere, per ogni super categoria, tutte le annotazioni e associarle alla super categoria corrispondente.

2 -  fare il fine-tuning del modello di CTRL tramite le annotazioni associate alla super categoria e generare del testo

    2.1 - modficare i control code durante la fase di fine tuning, in modo tale da inserire i nostri.  

3 - Valutare il testo generato tramite una versione modificata di TexyGen.

Dubbi: 
- bisogna usare le categorie o le super categorie? 

Commenti:

Al momento ho creato una demo che mostra le super categorie e prende le annotazioni di una categoria fissa.

CTRL sono riuscito solo a farlo girare in modalità low memory, ma è molto più inefficiente


**SIMONE:** guardare se la computazione della loss nella fase di fintuning sia corretta.

provare ad eseguire il finetuning con il trainer

la generazione genera parole a caso, non so se gli devo dare l'ultima parola generata o la frase generata fino a quel momento.


#Prepare environment

In [1]:
from google.colab import drive
from pycocotools.coco import COCO
import os
import subprocess
!pip install transformers

NUM_LINES_PER_ITERATION = 15

     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 3.3 MB 51.6 MB/s 
     |████████████████████████████████| 895 kB 59.7 MB/s 
     |████████████████████████████████| 636 kB 50.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
#https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb demo coco API
def init_COCO():           
  dataDir='/content/drive/MyDrive/Machine Learning/Progetto'
  dataType='val2017'
  modelDir = dataDir + '/model/' 
  instFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
  annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)
  
  coco=COCO(instFile) #modello con le foto e le categorie
  coco_ann=COCO(annFile) #modello con le annotazioni
  return coco,coco_ann

# Prepare Dataset

In [4]:
# display COCO categories and supercategories
def get_categories(coco):
  print('Getting categories...')
  cats = coco.loadCats(coco.getCatIds())
  nms=[cat['name'] for cat in cats]

  supernms = set([cat['supercategory'] for cat in cats])
  return nms

In [5]:
# load and display caption annotations by category
def get_annotation(coco,coco_ann,catNms = ['outdoor']):
  print('Getting annotations for {0}'.format(catNms))
  catIds = coco.getCatIds(catNms=catNms); #seleziono gli id delle categorie che voglio analizzare
  imgIds = coco.getImgIds(catIds=catIds); #ottengo gli id delle immagini delle categorie desiderate tramite gli id delle categorie
  annIds = coco_ann.getAnnIds(imgIds=imgIds); #ottengo gli id delle annotazioni relative alle immagini, tramite gli id delle immagini

  anns = coco_ann.loadAnns(annIds)
  return anns

# CTRL

## Huggingface

### Prepare model

In [6]:
# --- PREPARE model, tokenizer and device

import torch
class TextSet(torch.utils.data.Dataset):

  def __init__(self, text, labels = ""):
    print("TextSet inizializing...")
    self.labels = labels
    self.text = text

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    label = self.labels[idx]
    text = self.text[idx]
    sample = {"Text": text, "Class": label}
    return sample

def init_seq():
  print("Start init CTRLForSequenceClassification")
  from transformers import CTRLForSequenceClassification, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  model = CTRLForSequenceClassification.from_pretrained('ctrl', config = CTRLConfig(n_layer = 10, n_heads = 2), num_labels=1)
  model = model.to(device)
  return model, tokenizer, device

def init():
  print("Start init CTRLLMHeadModel")
  from transformers import CTRLLMHeadModel, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  model = CTRLLMHeadModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 9))
  return model, tokenizer, device

def init_tf():
  print("Start init TFCTRLLMHeadModel")
  if os.path.isfile("/content/drive/MyDrive/Machine Learning/Progetto/model/tf_model.h5") == False:
    from transformers import TFCTRLLMHeadModel, CTRLConfig, CTRLTokenizer
    import torch
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    tokenizer = CTRLTokenizer.from_pretrained('ctrl')
    model = TFCTRLLMHeadModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 12, n_heads = 3))
    model.save_pretrained("/content/drive/MyDrive/Machine Learning/Progetto/model")

  # TODO https://stackoverflow.com/questions/45275141/tensorflow-is-there-a-way-to-convert-a-frozen-graph-into-a-checkpoint-model
  # https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb/45466355#45466355
  # https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc
  saver = tf.train.Checkpoint()
  model = load_model('/content/drive/MyDrive/Machine Learning/Progetto/model/tf_model.hdf5', compile=False)
  sess = tf.compat.v1.keras.backend.get_session()
  save_path = saver.save("/content/drive/MyDrive/Machine Learning/Progetto/model/model.ckpt")
  return model, tokenizer, device


### Finetune

In [7]:
# PREPARE FINETUNING

from torch.utils.data import DataLoader
from transformers import AdamW, Trainer, TrainingArguments
import pandas as pd

def get_dataset(dataset_path,label):
  dataset = []
  labels = []
  infile = open(dataset_path,'r')
  for line in infile:
    line.replace('\\n','')
    line = 'outdoor: ' + line
    dataset.append(line)
    labels.append(label)
  infile.close()
  return dataset,labels

def format_dataset(annotations,start,category):
  dataset = []
  labels = []
  end = start + NUM_LINES_PER_ITERATION
  if len(annotations) < end:
    end = len(annotations)
  annotation = annotations[start:end ]
  text = category
  for ann in annotation:
    text += " " + ann['caption']
  dataset.append(text)
  labels.append(category)
  return dataset,(end + 1),labels

def fine(model,tokenizer,device,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1.txt",dataset_val_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Machine Learning/Progetto/results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,)
  
  dataset,labels = get_dataset(dataset_path,"outdoor")
  eval_dataset,labels = get_dataset(dataset_val_path,"outdoor")
  print(dataset)

  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,               # training dataset
    eval_dataset=eval_dataset            # evaluation dataset
    )
  trainer.train()

def prepare_dataset_v2(annotations, category, tokenizer):
  annotations_ = annotations[0:15000]
  annotations_train = []
  annotations_val = []
  for idx in iter(range(len(annotations_))):
    annotations_[idx] = category + " " + annotations_[idx]['caption']
    #annotations_[idx] = tokenizer.encode(annotations_[idx], add_special_tokens=False, return_tensors="pt")
    if idx % 15 == 0:
      annotations_val.append(tokenizer(annotations_[idx], padding="max_length", add_special_tokens=False, return_tensors="pt"))
    else:
      annotations_train.append(tokenizer(annotations_[idx], padding="max_length" ,add_special_tokens=False, return_tensors="pt"))
  return annotations_train,annotations_val

def finetune_v2(model,tokenizer,device,annotations,category = 'outdoor' ):#,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  print("start finetune")
  train_dataset = []
  model.to(device)
  len_annotation = len(annotations)
  print(len(annotations))
  len_annotation = 15000
  model.train()
  #Se non esiste il control code lo aggiungo
  encoded_category = tokenizer.encode(category, add_special_tokens=False)
  if not any(encoded_category[0] == x for x in tokenizer.control_codes.values()):
    tokenizer.control_codes[category] = encoded_category[0]
  #separo i due dataset
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  train_dataset,val_dataset = prepare_dataset_v2(annotations, category, tokenizer)

  training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Machine Learning/Progetto/results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=300,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/Machine Learning/Progetto/logs',            # directory for storing logs
    logging_steps=10,
  )
  print(train_dataset)
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset             # evaluation dataset
  )

  trainer.train()


def finetune(model,tokenizer,device,annotations,category = 'outdoor' ):#,dataset_path = "/content/drive/MyDrive/Machine Learning/Progetto/prova_1_val.txt"):
  print("start finetune")
  train_dataset = []
  model.to(device)
  len_annotation = len(annotations)
  print(len(annotations))
  #len_annotation = 15000
  model.train()
  #Se non esiste il control code lo aggiungo
  encoded_category = tokenizer.encode(category, add_special_tokens=False)
  if not any(encoded_category[0] == x for x in tokenizer.control_codes.values()):
    tokenizer.control_codes[category] = encoded_category[0]
  for epoch in range(3):
    print("EPOCH")
    index = 0
    while index < len_annotation:
      dataset,index,labels = format_dataset(annotations = annotations,start = index,category = category)
      print("{0} / {1}".format(index,len_annotation))
      text_labels_df = pd.DataFrame({'Text': dataset, 'Labels': labels})
      TD = TextSet(text_labels_df['Text'], text_labels_df['Labels'])
                            
      train_loader = DataLoader(TD, batch_size=1, shuffle=True)
      optim = AdamW(model.parameters(), lr=5e-5)
      for batch in train_loader:
        print(batch)
        batch_t = []
        optim.zero_grad()
        for b in batch["Text"]:
          batch_t = tokenizer(b, return_tensors="pt", truncation=True)
          print(batch_t)
        batch_t.to(device)
        outputs = model(**batch_t, labels=batch_t["input_ids"])
        loss = outputs[0]
        print(loss)
        loss.backward()
        optim.step()



  

### Generation

In [70]:
def generate(model,tokenizer,device,first_word = "outdoor "):
  model.eval()
  word = ""
  phrase = ""
  for k in range(10):
    phrase = phrase + word + " "
    print("phrase: " + phrase)
    inputs = tokenizer.encode_plus(first_word + word, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model(**inputs,labels=inputs["input_ids"])
    word =  decode_word(output = outputs)
    print("word:" + word[0])
  return outputs

def decode_word(output):
  x = ""
  loss = output.loss
  logits = output.logits
  print(logits)
  softmax = F.softmax(logits, dim = -1)
  index = torch.argmax(softmax,dim = -1)
  x = tokenizer.decode(index[0])
  return x

def prepare_ctrl_input(tokenizer, prompt_text):
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
  if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
      print(tokenizer.control_codes.values())
  return prompt_text

def generate_v2(model,tokenizer,device,first_word = "outdoor"):
  #controllo che il control code esista
  preprocessed_prompt_text = prepare_ctrl_input(tokenizer, first_word)
  tokenizer_kwargs = {}
  encoded_prompt = tokenizer.encode(
      preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", **tokenizer_kwargs
  )

  if encoded_prompt.size()[-1] == 0:
    input_ids = None
  else:
    input_ids = encoded_prompt
  output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length= 50 + len(encoded_prompt[0]),
        temperature=0.4,
        top_k=1,
        top_p=0.8,
        repetition_penalty=5.0,
        do_sample=True,
        num_return_sequences=1,
    )
  print(output_sequences)

  if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
      print(f"=== GENERATED SEQUENCE {generated_sequence_idx + 1} ===")
      generated_sequence = generated_sequence.tolist()

      # Decode text
      text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

      # Remove all text after the stop token
      text = text[: None]

      # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
      total_sequence = (
          first_word + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
      )

      generated_sequences.append(total_sequence)
      print(total_sequence)

### Execution

In [9]:
from torch.nn import functional as F
import torch
prepare_dataset = True
init_CTRL = True
finetune_CTRL_v2 = False
finetune_CTRL = True
generate_CTRL = True

if prepare_dataset == True:
  coco,coco_ann = init_COCO()
  cat = get_categories(coco = coco)
  ann = get_annotation(coco = coco, coco_ann = coco_ann) 
if init_CTRL == True:
  model,tokenizer,device = init()
if finetune_CTRL_v2 == True:
  finetune_v2(model = model,tokenizer = tokenizer,device = device, annotations = ann)
if finetune_CTRL == True:
  finetune(model = model,tokenizer = tokenizer,device = device, annotations = ann)
if generate_CTRL == True:
  output = generate_v2(model = model,tokenizer = tokenizer, device = device)


loading annotations into memory...
Done (t=2.17s)
creating index...
index created!
loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
Getting categories...
Getting annotations for ['outdoor']
Start init CTRLLMHeadModel


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Some weights of the model checkpoint at ctrl were not used when initializing CTRLLMHeadModel: ['transformer.h.32.ffn.0.bias', 'transformer.h.35.multi_head_attention.Wv.bias', 'transformer.h.47.multi_head_attention.Wq.weight', 'transformer.h.30.multi_head_attention.Wk.weight', 'transformer.h.9.layernorm1.weight', 'transformer.h.45.ffn.0.weight', 'transformer.h.29.multi_head_attention.Wq.weight', 'transformer.h.23.multi_head_attention.Wv.bias', 'transfo

Output streaming troncato alle ultime 5000 righe.
          59271,  22852,     75,    413,      1,    571,      4,   7048,    294,
              3,  53568,   1987,    571,  12685,      4,      5,    900,   7048,
           3685,    123,     29,     37,    737,      4,      0,   7048,   3685,
           1010,    571,     29,   7602,      4,      0,   7048,    125,     36,
          57343,     61,    571,     29,   2707,    125,     36,   9244,      4,
              5,   7048,     75,   8111,   2142,      4,      5,  50718,  11661,
            105,      5, 101759,   7884,    537,     63,      1,      5,   8111,
           2142,      4,      5,  50718,  11661,      5,    221,   8111,   8457,
           2045,      1,      5,  50718,  11661,     75,   2395,   8111,   8457,
              4,      5,    221,  94726,     75,   8111,   8457,      4,      5,
          50718,  11661,    125,    684, 171411,     75,   3610,   5839,   1213,
           2142,      4,      5,    609,    671,      4,   

In [71]:
output = generate_v2(model = model,tokenizer = tokenizer, device = device)

tensor([[ 10924,     75,    112,      2,    535,      4,      5,   3494,      8,
           1781,     27,  24213, 101306,     13,    571,     29,    912,     12,
              0,    434,      1, 108312,  11359,   4685,  19402,    762,  18883,
          59597,   2552,      9,     44,     43,     14,    575,  40442,   1010,
            156,   1351,    269,    166,     59,     18,     40,   4720,     61,
            164,   2142,    132,    105,     35,   6365]], device='cuda:0')
=== GENERATED SEQUENCE 1 ===
outdoor A man and woman in a dress is holding an umbrella umbrella. The women are playing with the game of golf. Golf clubs clubs, club clubs... that has been on top thereof. Two people standing near each other at one another. two men sitting down under their respective


# Texygen evaluation

**Where the execution being stopped raising errors concerned about tensorflow,  replace `import tensorflow as tf` with `import tensorflow.compat.v1 as tf` and adding `tf.disable_v2_behavior()` before the code**





In [11]:
def download_texygen():
  !pip install colorama

  import nltk
  nltk.download('punkt')
  project_path = '/content/drive/MyDrive/Machine Learning/Progetto'
  os.chdir(project_path)
  if not os.path.isdir('/content/drive/MyDrive/Machine Learning/Progetto/Texygen'):
    !git clone https://github.com/geek-ai/Texygen.git

def run_texygen(gan = "mle", training = "oracle", data = "./data/image_coco.txt"):
  texygen_path = '/content/drive/MyDrive/Machine Learning/Progetto/Texygen'
  os.chdir(texygen_path)
  !python3 main.py -g $gan -t $training -d $data

def plot_results_texygen(gan = "mle", training = "oracle", data = "./data/image_coco.txt"): 
  import pandas as pd
  import matplotlib.pyplot as plt

  base_path = "/content/drive/MyDrive/Machine Learning/Progetto/Texygen/experiment-log-" + gan
  suffix_path = "-" + training + ".csv" if training != "oracle" else ".csv"
  result_file_path = base_path + suffix_path

  print("READING FILE: " + result_file_path)
  print("")
  results = pd.read_csv(result_file_path)
  results.dropna(axis = "columns", inplace = True)
  print(results)

  epochs = results.iloc[:, 0]
  metrics = {}
  for index in range(1, len(results.columns)):  
    metrics["metric{0}".format(index)] = results.iloc[:, index]

  with plt.style.context("Solarize_Light2"):
    for key, value in metrics.items():
      plt.plot(epochs, value, "-b", marker="o", label = value.name)
      plt.legend()

  plt.show()

In [12]:
download_texygen()
run_texygen("leakgan", "real")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'Texygen'...
remote: Enumerating objects: 888, done.
remote: Total 888 (delta 0), reused 0 (delta 0), pack-reused 888
Receiving objects: 100% (888/888), 21.85 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (537/537), done.
Checking out files: 100% (121/121), done.
2021-07-27 13:01:06.566848: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "main.py", line 6, in <module>
    from models.gsgan.Gsgan import Gsgan
  File "/content/drive/MyDrive/Machine Learning/Progetto/Texygen/models/gsgan/Gsgan.py", line 4, in <module>
    from models.Gan import Gan
  File "/content/drive/MyDrive/Machine Learning/Progetto/Texygen/models/Gan.py", line 3, in <module>
    from utils.utils import init_sess
  File "/content/drive/MyDrive/Machine Learning/Progetto/Texygen/utils/utils

In [13]:
plot_results_texygen("seqgan", "real")

READING FILE: /content/drive/MyDrive/Machine Learning/Progetto/Texygen/experiment-log-seqgan-real.csv



FileNotFoundError: ignored

# No texygen evaluation

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

print(output.logits)

##API e salesforce

### Prepare model

In [ ]:
from shutil import copyfile
import gc

# --- INITIALIZING SOME CONSTANT ----

lowerMemory = False
mixed = False
project_path = "/content/drive/MyDrive/Machine Learning/Progetto/"
ctrl_path = project_path + "ctrl/"
training_path = ctrl_path + "training_utils/"
pretrained_model_dir = "model/"
model_name = "model.ckpt-684000.data-00000-of-00001"
model_dir = "seqlen256_36layers_v0.ckpt/"  

def Patch_Keras(old_path = ctrl_path):
  os.chdir(ctrl_path)
  !patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch
  os.chdir(old_path)

def init_no_API():
  os.chdir(project_path) #%cd project_path
  !nvidia-smi
  !pip2 install tensorflow-gpu==1.14 #installo le dipendenze
  prova = training_path + "prova1.txt"
  !rm -rf ctrl
  !rm -rf prova
  !git clone https://github.com/salesforce/ctrl #clono CTRL
  os.chdir(ctrl_path) #%cd ctrl_path
  if lowerMemory == True:
    Path_Keras(ctrl_path)
    !git checkout lower_memory
  else:
    !pip install torch
    !git checkout master
  !pip2 install fastBPE

  if mixed == False:
    if os.path.isfile(project_path + model_dir + model_name) == False:
      os.chdir(project_path)#%cd project_path
      !pip2 install gsutil #gsutil is a Python application that lets you access Cloud Storage from the command line. You can use gsutil to do a wide range of bucket and object management tasks
      !gsutil -m cp -r gs://sf-ctrl/seqlen256_36layers_v0.ckpt/ .
      #!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt . #48 layer
      #()
  else:
    print("MIXED @@@@@@@@@@@@@@@@")
    #init_mixed()

def generate_no_API():
  gc.collect()
  os.chdir(ctrl_path)#%cd ctrl_path
  if lowerMemory == True:
    !python2 generation.py --model ../seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001
  else:
    !python2 pytorch_generation.py --temperature 0.5 --model ../seqlen256_v1.ckpt/model.ckpt-684000.data-00000-of-00001

def init_API():
  !pip install transformers
  from transformers import TFCTRLLMHeadModel, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  model = TFCTRLLMHeadModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 10))
  
  return model,tokenizer,device

def generate_API(model,tokenizer,device):
  inputs = tokenizer("Politics Title:", return_tensors="tf")
  outputs = model(inputs)
  return outputs

def finetune_CTRL_API():
  #farò il finetune e salverò il modello su drive, cancellando la cartella del API = false
  #nel metodo API = False dovrò cancellare questo modello
  i = 50

def Create_tf_records(file_list = [("outdoor","prova_1.txt")]): 
  #avrò una lista con [(nome_label,path_file),...]. dovrò leggere path_file da google drive e produrrò una lista con il path al file contenente il tf record
  Patch_Keras(training_path)  
  #os.system("wget -O moby_dick.txt https://www.gutenberg.org/files/2701/2701-0.txt") #DEBUG, per le prove finchè non avrò la lista
  #mi salva i record in moby_dick.txt.tfrecords
  for (control_f,path_f) in file_list:
    copyfile(project_path + path_f, training_path + path_f)
    path_file = training_path + path_f + ".tfrecords"
    if os.path.isfile(path_file) == False:
      #se non esiste il file tfrecords devo crearlo
      command_create_tf = "python2 make_tf_records.py --text_file {0} --control_code {1} --sequence_len 256".format(path_f,control_f)
      subprocess.check_output(command_create_tf,shell=True)
      print("Creato file con control code {0}".format(control_f))

def finetune_CTRL_no_API(file_list = []):
  Create_tf_records()
  #train_cmd = "python2 training.py --model_dir {} --iterations 1".format(project_path.replace(" ","\\ ") + model_dir)
  print("START_TRAINING")
  gc.collect()
  #subprocess.check_output(train_cmd,shell = True)
  !python2 training.py --model_dir /content/drive/MyDrive/Machine\ Learning/Progetto/seqlen256_36layers_v0.ckpt/ --iterations 1

### Text generation

In [ ]:
use_api = True
finetune_CTRL = True
generate_CTRL = False
if use_api == False:
  init_no_API()
  if finetune_CTRL == True:
    finetune_CTRL_no_API()
  if generate_CTRL == True:
    generate_no_API()
else:
  model,tokenizer,device = init_API()
  output = generate_API(model = model,tokenizer = tokenizer, device = device)